# 1. Užduotis

Duomenys iš pirmos užduoties 2) punkto, t.y. stebėtas a.d., turintis Veibulo skirstinį su
parametrais $\eta$ ir $\nu$, t. y. a. d. $T$ pasiskirstymo funkcija yra

$$
F(t; \eta, \nu) = 1 - \exp{\{ - {(t / \eta)}^\nu \}},
$$

gauta didumo $n = 100$ paprastoji atsitiktinė imtis.

In [1]:
set.seed(314)

n <- 100
eta <- 2 # laisvai pasirinkti 
nu <- 2

weibull_inverse_cdf <- function(theta) {
    function(probs) {
        theta[1] * (-1 * log(1 - probs)) ^ (1 / theta[2])
    }
}

observ <- weibull_inverse_cdf(c(eta, nu))(runif(n))

**Pastaba:** Taškiniai įverčiai buvo rasti pirmoje užduotyje.

In [2]:
# Analogiškai pirmosios dalies antrai užduočiai:
gamma <- 0.5772156 
sigma_hat_moment <- function(x) { sqrt(6 * var(x)) / pi }
mu_hat_moment <- function(x) { mean(x) + gamma * sigma_hat_moment(x) }

In [3]:
(start_eta <- exp(mu_hat_moment(log(observ))))
(start_nu <- 1 / sigma_hat_moment(log(observ)))

[1] 1.820739

[1] 1.919287

In [4]:
weibull_loglik <- function(observ) {
    log_sum <- sum(log(observ))
    n <- length(observ)

    function(theta) {
        eta <- theta[1]
        nu <- theta[2]
        
        n * (log(eta) - log(nu))
        - (1 / nu^eta) * sum(observ^eta)
        + (eta - 1) * (log_sum - n * log(nu))
    }
}

optim_optimiser <- function(f, start) {
    mle <- stats::optim(
        start, f,
        method = "L-BFGS-B",
        hessian = TRUE
    )
    r <- list(
        estimates = mle$par,
        hessian = mle$hessian
    )
    return(r)
}

mle_estimator <- function(loglikelihood, optimiser) {
    function(data, start) {
        optimiser(loglikelihood(data), start)
    }
}

weibull_mle_estimator <- mle_estimator(weibull_loglik, optim_optimiser)

In [5]:
(theta_hat_mle <- weibull_mle_estimator(observ, c(start_eta, start_nu)))

-8.881784e-10,-52.10270
-5.210270e+01,22.28055


a) Raskite stebėtą Fišerio informacinę matricą.

In [11]:
(fishers_info <- -1 * theta_hat_mle$hessian)

8.881784e-10,52.10270
5.210270e+01,-22.28055


b) Raskite parametrų pasikliovimo intervalus. Rezultatus (tikros parametrų reikšmės,
taškiniai įverčiai ir pasikliovimo intervalai) pateikite duomenų lentelėje (dataframe).

Pasinaudosime Didžiausio Tikėtinumo įvertinių asimptotiniu normalumu:

In [30]:
mle_asymptotic_ci <- function(mle, sample_size) {
    mle_estimates <- mle$estimates
    fishers_info <- -1 * mle$hessian
    
    # Precalculate:
    fishers_inverse <- solve(fishers_info)
    sqrt_size <- sqrt(sample_size)

    function(confidence) {
        i <- 1
        alpha_half <- (1 - confidence) / 2 

        lapply(mle_estimates, function(e) {
            variance <- fishers_inverse[i, i]
            bias <- sqrt(variance) * qnorm(alpha_half, lower.tail = FALSE) / sqrt_size
            i <- i + 1

            ci <- list(
                lo = e - bias,
                hi = e + bias
            )
                
            return(ci)
        })
    }
}

In [54]:
weibull_ci <- mle_asymptotic_ci(theta_hat_mle, n)
ci95 <- weibull_ci(0.87)

In [55]:
data.frame(
    name = c("eta", "nu"),
    actual = c(eta, nu),
    lo_95 = c(ci95[[1]]$lo, ci95[[2]]$lo),
    ml_estimate = c(theta_hat_mle$estimates[1], theta_hat_mle$estimates[2]),
    hi_95 = c(ci95[[1]]$hi, ci95[[2]]$hi) 
)

name,actual,lo_95,ml_estimate,hi_95
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
eta,2,1.807022,1.820739,1.834456
nu,2,1.905570,1.919287,1.933003


c) Raskite tikimybės, kad stebėtas a.d. įgis reikšmę didesnę už $t$, pasikliovimo intervalą.
Rezultatus ($t$, tikra tikimybės reikšmė, tikimybės įvertis, pasikliovimo lygmuo, pasikliovimo
intervalas) pateikite duomenų lentelėje (dataframe).

d) Raskite intervalinį medianos įvertį. Rezultatus (tikra medianos reikšmė, taškinis
įvertis, pasikliovimo lygmuo ir pasikliovimo intervalas) pateikite duomenų lentelėje (dataframe).

e) Raskite $p$-ojo kvantilio pasikliovimo intervalą. Rezultatus (kvantilio lygmuo $p$, tikra
kvantilio reikšmė, taškinis įvertis, pasikliovimo lygmuo ir pasikliovimo intervalas) pateikite
duomenų lentelėje (dataframe)